## Идеи

1. Необходимо классифицировать изображения на 12 классов. Используем простую сверточную сеть с двумя полносвязными слоями. На выходе - слой размера 12 с softmax. На входе - тензор батча изобржаний
2. Изображения очень похожи друг на друга. В центре - побег растения, фон - почва (камни, замля и т.д.)
3. Изображения не стандартного формата (вроде). Необходимо привести к одному масштабу. 
4. Так как фото сделаны сверху - нет строгой ориентации побега (однако, он центрирован) и нет строгого отношения размер изображения/размер побега. Поэтому точно классной идеей будет применить аугментацию изображений поворотом и масштабированием. Возможно, также сдвигом. ImageDataGenerator
5. В качестве доп. фичей можно применить гистограмму цветов в области побега
6. Обучить несколько независимых сетей на частях данных (допустим, 4), делать стекинг на тестовой выборке из их предсказаний. Сам стекинг не обучать (для скорости)
7. Попробовать дообучать несколько слоев ResNet (хотя, эта сеть обучена для слишком большого количества изображений)
8. После обучения смотреть матрицу ошибок, выделять дополнительно то, что можно дообучить в случае больших ошибок сети
9. Подумать над размером и стратификацией батча (видел батчи по 16-20 фото, но у нас 12 классов. Так, есть большая вероятность не попадания нескольких классов в батч)

## imports

In [1]:
import pandas as pd
import numpy as np
import os
import imageio

from keras.utils import plot_model
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Maximum
from keras.layers import ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize as imresize
from tqdm import tqdm
import os
import numpy as np
import cv2
from glob import glob
from sklearn.decomposition import PCA

# from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# global vars

In [2]:
BATCH_SIZE = 16
EPOCHS = 50
RANDOM_STATE = 5
CLASS = {
    'Black-grass': 0,
    'Charlock': 1,
    'Cleavers': 2,
    'Common Chickweed': 3,
    'Common wheat': 4,
    'Fat Hen': 5,
    'Loose Silky-bent': 6,
    'Maize': 7,
    'Scentless Mayweed': 8,
    'Shepherds Purse': 9,
    'Small-flowered Cranesbill': 10,
    'Sugar beet': 11
}

INV_CLASS = {
    0: 'Black-grass',
    1: 'Charlock',
    2: 'Cleavers',
    3: 'Common Chickweed',
    4: 'Common wheat',
    5: 'Fat Hen',
    6: 'Loose Silky-bent',
    7: 'Maize',
    8: 'Scentless Mayweed',
    9: 'Shepherds Purse',
    10: 'Small-flowered Cranesbill',
    11: 'Sugar beet'
}

# Создание сети

In [3]:
def dense_set(inp_layer, n, activation, drop_rate=0.):
    dp = Dropout(drop_rate)(inp_layer)
    dns = Dense(n)(dp)
    bn = BatchNormalization(axis=-1)(dns)
    act = Activation(activation=activation)(bn)
    return act

In [4]:
def conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = Conv2D(filters=feature_map, kernel_size=kernel_size, strides=strides)(zp)
    bn = BatchNormalization(axis=3)(conv)
    act = LeakyReLU(1/10)(bn)
    return act

In [5]:
def get_model():
    inp_img = Input(shape=(51, 51, 10))

    # 51
    conv1 = conv_layer(inp_img, 64, zp_flag=False)
    conv2 = conv_layer(conv1, 64, zp_flag=False)
    mp1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv2)
    # 23
    conv3 = conv_layer(mp1, 128, zp_flag=False)
    conv4 = conv_layer(conv3, 128, zp_flag=False)
    mp2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv4)
    # 9
    conv7 = conv_layer(mp2, 256, zp_flag=False)
    conv8 = conv_layer(conv7, 256, zp_flag=False)
    conv9 = conv_layer(conv8, 256, zp_flag=False)
    mp3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv9)
    # 1
    # dense layers
    flt = Flatten()(mp3)
    ds1 = dense_set(flt, 128, activation='tanh')
    out = dense_set(ds1, 12, activation='softmax')

    model = Model(inputs=inp_img, outputs=out)

    
#     mypotim = Adam(lr=2 * 1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    mypotim = SGD(lr=1 * 1e-4, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                   optimizer=mypotim,
                   metrics=['accuracy'])
    model.summary()
    return model

In [6]:
def get_callbacks(filepath, patience=5):
    lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=1e-5, patience=patience, verbose=1)
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [lr_reduce, msave]

In [7]:
# Архитектура MNIST, не применяю тут
def gen_model():
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(51, 51, 3)),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(rate=0.25),
        Flatten(),
        Dense(1024, activation='relu'),
        Dense(12, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
get_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 51, 51, 10)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 49, 49, 64)        5824      
_________________________________________________________________
batch_normalization_1 (Batch (None, 49, 49, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 47, 47, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 47, 47, 64)        0         
__________

## Методы для обучения и предсказания

In [54]:
def train_model(img, target):
#     callbacks = get_callbacks(filepath='model_weight_Adam.hdf5', patience=6)
    callbacks = get_callbacks(filepath='model_weight_SGD.hdf5', patience=5)
    gmodel = get_model()
    gmodel.load_weights(filepath='night_weights.hdf5')
    x_train, x_valid, y_train, y_valid = train_test_split(
                                                        img,
                                                        target,
                                                        shuffle=True,
                                                        train_size=0.8,
                                                        random_state=RANDOM_STATE
                                                        )
    gen = ImageDataGenerator(
            rotation_range=180.,
            width_shift_range=0.4,
            height_shift_range=0.4,
            zoom_range=0.5,
            vertical_flip=True
    )
    gmodel.fit_generator(gen.flow(x_train, y_train,batch_size=BATCH_SIZE),
               steps_per_epoch=10*len(x_train)/BATCH_SIZE,
               epochs=EPOCHS,
               verbose=1,
               shuffle=True,
               validation_data=(x_valid, y_valid),
               callbacks=callbacks
               )
    gmodel.save_weights('night_weights.hdf5')
#     

In [60]:
def test_model(img, label):
    gmodel = get_model()
#     gmodel.load_weights(filepath='model_weight_Adam.hdf5')
#     gmodel.load_weights(filepath='night_weights.hdf5')
    gmodel.load_weights(filepath='model_weight_SGD.hdf5')
    prob = gmodel.predict(img, verbose=1)
    pred = prob.argmax(axis=-1)
    sub = pd.DataFrame({"file": label,
                         "species": [INV_CLASS[p] for p in pred]})
    sub.to_csv("sub.csv", index=False, header=True)

# чтение и предобработка данных

In [33]:
def img_reshape(img):
    img = imresize(img, (51, 51, 10))
    return img

In [11]:
def img_label(path):
    return str(str(path.split('\\')[-1]))

In [12]:
def img_class(path):
    return str(path.split('\\')[-2])

In [23]:
def fill_dict(paths, some_dict):
    text = ''
    if 'train' in paths[0]:
        text = 'Start fill train_dict'
    elif 'test' in paths[0]:
        text = 'Start fill test_dict'

    for p in tqdm(paths, ascii=True, ncols=85, desc=text):
        img = cv2.imread(p)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)    
        mask = cv2.inRange(hsv, (24, 50, 0), (55, 255, 140))
        img_m = cv2.bitwise_and(img, img, mask=mask)
        img_tot = np.concatenate([img, hsv, mask[..., np.newaxis], img_m], axis = -1)
        
#         img = np.concatenate([bgr, hsv, mask[..., np.newaxis]], axis=-1)
        
        img = img_reshape(img_tot)
        some_dict['image'].append(img)
        some_dict['label'].append(img_label(p))
        if 'train' in paths[0]:
            some_dict['class'].append(img_class(p))
    return some_dict

In [24]:
def reader():
    file_ext = []
    train_path = []
    test_path = []

    for root, dirs, files in os.walk('../../Data'):
        if dirs != []:
            print('Root:\n'+str(root))
            print('Dirs:\n'+str(dirs))
        else:
            for f in files:
                ext = os.path.splitext(str(f))[1][1:]

                if ext not in file_ext:
                    file_ext.append(ext)

                if 'train' in root:
                    path = os.path.join(root, f)
                    train_path.append(path)
                elif 'test' in root:
                    path = os.path.join(root, f)
                    test_path.append(path)
    train_dict = {
        'image': [],
        'label': [],
        'class': []
    }
    test_dict = {
        'image': [],
        'label': []
    }

    train_dict = fill_dict(train_path, train_dict)
    test_dict = fill_dict(test_path, test_dict)
    return train_dict, test_dict

## Собственно, чтение данных в словарь

In [34]:
train_dict, test_dict = reader()

Root:
../../Data
Dirs:
['NLP-HW3', 'test', 'train']
Root:
../../Data\NLP-HW3
Dirs:
['names']
Root:
../../Data\train
Dirs:
['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']


Start fill train_dict:   0%|                                | 0/4750 [00:00<?, ?it/s]C:\Anaconda\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Start fill test_dict: 100%|########################| 794/794 [00:23<00:00, 33.32it/s]


## Составление необходимых ndarray, передаваемых в модель

In [36]:
X_train = np.array(train_dict['image'])
y_train = to_categorical(np.array([CLASS[l] for l in train_dict['class']]))

In [37]:
X_test = np.array(test_dict['image'])
label = test_dict['label']

In [38]:
X_train[0][0][0]

array([0.11951625, 0.20570897, 0.32363005, 0.05098039, 0.633757  ,
       0.32363005, 0.        , 0.        , 0.        , 0.        ])

## Обучение модели

In [57]:
train_model(X_train, y_train)

C:\Anaconda\lib\site-packages\keras\callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 51, 51, 10)        0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 49, 49, 64)        5824      
_________________________________________________________________
batch_normalization_82 (Batc (None, 49, 49, 64)        256       
_________________________________________________________________
leaky_re_lu_64 (LeakyReLU)   (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 47, 47, 64)        36928     
_________________________________________________________________
batch_normalization_83 (Batc (None, 47, 47, 64)        256       
_________________________________________________________________
leaky_re_lu_65 (LeakyReLU)   (None, 47, 47, 64)        0         
__________

C:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\Anaconda\lib\site-packages\keras_preprocessing\image.py:1404: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (3800, 51, 51, 10) (10 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


Epoch 1/15


 239/2375 [==>...........................] - ETA: 1:49:19 - loss: 0.1370 - acc: 0.93 - ETA: 55:49 - loss: 0.1872 - acc: 0.9062 - ETA: 38:05 - loss: 0.1668 - acc: 0.91 - ETA: 29:06 - loss: 0.1546 - acc: 0.92 - ETA: 23:42 - loss: 0.1561 - acc: 0.92 - ETA: 20:06 - loss: 0.1779 - acc: 0.91 - ETA: 17:32 - loss: 0.1721 - acc: 0.91 - ETA: 15:35 - loss: 0.1758 - acc: 0.92 - ETA: 12:48 - loss: 0.1622 - acc: 0.93 - ETA: 11:50 - loss: 0.1656 - acc: 0.93 - ETA: 11:00 - loss: 0.1554 - acc: 0.93 - ETA: 9:40 - loss: 0.1436 - acc: 0.9420 - ETA: 9:09 - loss: 0.1696 - acc: 0.937 - ETA: 8:42 - loss: 0.1763 - acc: 0.933 - ETA: 8:19 - loss: 0.1708 - acc: 0.933 - ETA: 7:58 - loss: 0.1634 - acc: 0.937 - ETA: 7:39 - loss: 0.1576 - acc: 0.940 - ETA: 7:22 - loss: 0.1528 - acc: 0.943 - ETA: 6:51 - loss: 0.1441 - acc: 0.946 - ETA: 6:25 - loss: 0.1440 - acc: 0.945 - ETA: 6:14 - loss: 0.1428 - acc: 0.947 - ETA: 5:54 - loss: 0.1417 - acc: 0.949 - ETA: 5:46 - loss: 0.1434 - acc: 0.948 - ETA: 5:29 - loss: 0.1451 - acc

...

1674/2375 [====================>.........] - ETA: 55s - loss: 0.1369 - acc: 0.95 - ETA: 55s - loss: 0.1368 - acc: 0.95 - ETA: 55s - loss: 0.1369 - acc: 0.95 - ETA: 55s - loss: 0.1369 - acc: 0.95 - ETA: 55s - loss: 0.1369 - acc: 0.95 - ETA: 55s - loss: 0.1370 - acc: 0.95 - ETA: 55s - loss: 0.1369 - acc: 0.95 - ETA: 55s - loss: 0.1368 - acc: 0.95 - ETA: 55s - loss: 0.1368 - acc: 0.95 - ETA: 55s - loss: 0.1368 - acc: 0.95 - ETA: 55s - loss: 0.1368 - acc: 0.95 - ETA: 55s - loss: 0.1367 - acc: 0.95 - ETA: 55s - loss: 0.1367 - acc: 0.95 - ETA: 54s - loss: 0.1367 - acc: 0.95 - ETA: 54s - loss: 0.1366 - acc: 0.95 - ETA: 54s - loss: 0.1366 - acc: 0.95 - ETA: 54s - loss: 0.1366 - acc: 0.95 - ETA: 54s - loss: 0.1366 - acc: 0.95 - ETA: 54s - loss: 0.1366 - acc: 0.95 - ETA: 54s - loss: 0.1366 - acc: 0.95 - ETA: 54s - loss: 0.1367 - acc: 0.95 - ETA: 54s - loss: 0.1366 - acc: 0.95 - ETA: 54s - loss: 0.1365 - acc: 0.95 - ETA: 54s - loss: 0.1364 - acc: 0.95 - ETA: 54s - loss: 0.1363 - acc: 0.95 - ETA: 

1893/2375 [======================>.......] - ETA: 42s - loss: 0.1366 - acc: 0.95 - ETA: 42s - loss: 0.1366 - acc: 0.95 - ETA: 42s - loss: 0.1365 - acc: 0.95 - ETA: 42s - loss: 0.1365 - acc: 0.95 - ETA: 42s - loss: 0.1367 - acc: 0.95 - ETA: 42s - loss: 0.1368 - acc: 0.95 - ETA: 42s - loss: 0.1368 - acc: 0.95 - ETA: 42s - loss: 0.1370 - acc: 0.95 - ETA: 42s - loss: 0.1369 - acc: 0.95 - ETA: 42s - loss: 0.1368 - acc: 0.95 - ETA: 42s - loss: 0.1368 - acc: 0.95 - ETA: 42s - loss: 0.1367 - acc: 0.95 - ETA: 42s - loss: 0.1367 - acc: 0.95 - ETA: 42s - loss: 0.1366 - acc: 0.95 - ETA: 41s - loss: 0.1368 - acc: 0.95 - ETA: 41s - loss: 0.1368 - acc: 0.95 - ETA: 41s - loss: 0.1368 - acc: 0.95 - ETA: 41s - loss: 0.1369 - acc: 0.95 - ETA: 41s - loss: 0.1369 - acc: 0.95 - ETA: 41s - loss: 0.1368 - acc: 0.95 - ETA: 41s - loss: 0.1368 - acc: 0.95 - ETA: 41s - loss: 0.1368 - acc: 0.95 - ETA: 41s - loss: 0.1370 - acc: 0.95 - ETA: 41s - loss: 0.1369 - acc: 0.95 - ETA: 41s - loss: 0.1369 - acc: 0.95 - ETA: 

2112/2375 [=========================>....] - ETA: 29s - loss: 0.1379 - acc: 0.95 - ETA: 29s - loss: 0.1380 - acc: 0.95 - ETA: 29s - loss: 0.1383 - acc: 0.95 - ETA: 29s - loss: 0.1386 - acc: 0.95 - ETA: 29s - loss: 0.1385 - acc: 0.95 - ETA: 29s - loss: 0.1386 - acc: 0.95 - ETA: 28s - loss: 0.1386 - acc: 0.95 - ETA: 28s - loss: 0.1387 - acc: 0.95 - ETA: 28s - loss: 0.1387 - acc: 0.95 - ETA: 28s - loss: 0.1387 - acc: 0.95 - ETA: 28s - loss: 0.1386 - acc: 0.95 - ETA: 28s - loss: 0.1386 - acc: 0.95 - ETA: 28s - loss: 0.1385 - acc: 0.95 - ETA: 28s - loss: 0.1385 - acc: 0.95 - ETA: 28s - loss: 0.1385 - acc: 0.95 - ETA: 28s - loss: 0.1385 - acc: 0.95 - ETA: 28s - loss: 0.1385 - acc: 0.95 - ETA: 28s - loss: 0.1384 - acc: 0.95 - ETA: 28s - loss: 0.1384 - acc: 0.95 - ETA: 28s - loss: 0.1384 - acc: 0.95 - ETA: 28s - loss: 0.1386 - acc: 0.95 - ETA: 27s - loss: 0.1385 - acc: 0.95 - ETA: 27s - loss: 0.1384 - acc: 0.95 - ETA: 27s - loss: 0.1384 - acc: 0.95 - ETA: 27s - loss: 0.1384 - acc: 0.95 - ETA: 

2347/2375 [============================>.] - ETA: 15s - loss: 0.1376 - acc: 0.95 - ETA: 15s - loss: 0.1376 - acc: 0.95 - ETA: 15s - loss: 0.1376 - acc: 0.95 - ETA: 15s - loss: 0.1375 - acc: 0.95 - ETA: 15s - loss: 0.1375 - acc: 0.95 - ETA: 15s - loss: 0.1374 - acc: 0.95 - ETA: 15s - loss: 0.1375 - acc: 0.95 - ETA: 15s - loss: 0.1375 - acc: 0.95 - ETA: 15s - loss: 0.1375 - acc: 0.95 - ETA: 15s - loss: 0.1375 - acc: 0.95 - ETA: 15s - loss: 0.1375 - acc: 0.95 - ETA: 15s - loss: 0.1374 - acc: 0.95 - ETA: 15s - loss: 0.1374 - acc: 0.95 - ETA: 15s - loss: 0.1374 - acc: 0.95 - ETA: 15s - loss: 0.1374 - acc: 0.95 - ETA: 14s - loss: 0.1374 - acc: 0.95 - ETA: 14s - loss: 0.1374 - acc: 0.95 - ETA: 14s - loss: 0.1375 - acc: 0.95 - ETA: 14s - loss: 0.1374 - acc: 0.95 - ETA: 14s - loss: 0.1373 - acc: 0.95 - ETA: 14s - loss: 0.1374 - acc: 0.95 - ETA: 14s - loss: 0.1374 - acc: 0.95 - ETA: 14s - loss: 0.1374 - acc: 0.95 - ETA: 14s - loss: 0.1374 - acc: 0.95 - ETA: 14s - loss: 0.1374 - acc: 0.95 - ETA: 

2375/2375 [==============================] - ETA: 1s - loss: 0.1387 - acc: 0.952 - ETA: 1s - loss: 0.1387 - acc: 0.952 - ETA: 1s - loss: 0.1388 - acc: 0.952 - ETA: 1s - loss: 0.1387 - acc: 0.952 - ETA: 1s - loss: 0.1387 - acc: 0.952 - ETA: 1s - loss: 0.1388 - acc: 0.952 - ETA: 1s - loss: 0.1387 - acc: 0.952 - ETA: 1s - loss: 0.1387 - acc: 0.952 - ETA: 1s - loss: 0.1387 - acc: 0.952 - ETA: 1s - loss: 0.1387 - acc: 0.952 - ETA: 0s - loss: 0.1386 - acc: 0.952 - ETA: 0s - loss: 0.1387 - acc: 0.952 - ETA: 0s - loss: 0.1387 - acc: 0.952 - ETA: 0s - loss: 0.1388 - acc: 0.952 - ETA: 0s - loss: 0.1387 - acc: 0.952 - ETA: 0s - loss: 0.1387 - acc: 0.952 - ETA: 0s - loss: 0.1387 - acc: 0.952 - ETA: 0s - loss: 0.1387 - acc: 0.952 - ETA: 0s - loss: 0.1387 - acc: 0.952 - ETA: 0s - loss: 0.1387 - acc: 0.952 - ETA: 0s - loss: 0.1386 - acc: 0.952 - ETA: 0s - loss: 0.1386 - acc: 0.952 - ETA: 0s - loss: 0.1386 - acc: 0.952 - ETA: 0s - loss: 0.1386 - acc: 0.952 - ETA: 0s - loss: 0.1386 - acc: 0.952 - 144s 

In [61]:
test_model(X_test, label)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 51, 51, 10)        0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 49, 49, 64)        5824      
_________________________________________________________________
batch_normalization_100 (Bat (None, 49, 49, 64)        256       
_________________________________________________________________
leaky_re_lu_78 (LeakyReLU)   (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 47, 47, 64)        36928     
_________________________________________________________________
batch_normalization_101 (Bat (None, 47, 47, 64)        256       
_________________________________________________________________
leaky_re_lu_79 (LeakyReLU)   (None, 47, 47, 64)        0         
__________

In [28]:
# 0.6801